COPYRIGHT © 2018 Kiran Arun <kironni@gmail.com>

### Setup

In [1]:
# setup
!rm -r Reinforcement_Learning-101-challenge
!git clone https://github.com/KiranArun/Reinforcement_Learning-101-challenge.git 1>/dev/null
!bash Reinforcement_Learning-101-challenge/scripts/setup.sh

rm: cannot remove 'Reinforcement_Learning-101-challenge': No such file or directory
Cloning into 'Reinforcement_Learning-101-challenge'...
remote: Counting objects: 103, done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 103 (delta 57), reused 65 (delta 27), pack-reused 0
Receiving objects: 100% (103/103), 56.33 KiB | 8.05 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [2]:
import os
!bash /content/Reinforcement_Learning-101-demo/scripts/kill-novnc.sh >/dev/null
!sleep 2
os.environ['PATH'] += "${PATH}:/opt/VirtualGL/bin:/opt/TurboVNC/bin"
!timeout 5 /opt/websockify/run 5901 --web=/opt/noVNC --wrap-mode=ignore -- /opt/TurboVNC/bin/vncserver :1 -securitytypes otp -otp -noxstartup 2>/dev/null
get_ipython().system_raw('python /opt/websockify/run 5901 --web=/opt/noVNC --wrap-mode=ignore -- /opt/TurboVNC/bin/vncserver :1 -securitytypes otp -otp -noxstartup > /content/.vnc/stdout 2>&1 &')
get_ipython().system_raw('/content/ngrok http -config=/content/config.yml 5901 &')
!curl -s http://localhost:4045/api/tunnels | python3 -c "import sys, json; print('XWindow Link:', json.load(sys.stdin)['tunnels'][0]['public_url'])"
get_ipython().system_raw('DISPLAY=:1 openbox &')
!grep "one-time password:" /content/.vnc/stdout

XWindow Link: http://2fc8fe76.ngrok.io
Full control one-time password: 38692308


In [0]:
get_ipython().system_raw('/usr/bin/Xvfb :99 -screen 0 640x480x24 &')
get_ipython().system_raw('/usr/bin/x11vnc -rfbport 5902 -forever -display :99 &')
get_ipython().system_raw('DISPLAY=:1 /usr/bin/vncviewer localhost:5902 &')
get_ipython().system_raw('DISPLAY=:99 xterm &')

# RL Challenge

In [4]:
# importing libraries
import numpy as np
import matplotlib.pyplot as plt
import gym

import keras
import keras.layers as layers
from rl import agents,memory,policy

Using TensorFlow backend.


### Hyperparameters

In [0]:
# Hyperparams

# number of steps to keep in experience buffer
memory_limit = 400

# discount value
gamma = 0.99

# how much to update target graph
target_model_update = 1e-2

# learning rate
learning_rate = 1e-2

# number of steps to sample from buffer to train on
batch_size = 32

In [6]:
# DO NOT EDIT

# setting environment
env = gym.make('CartPole-v1')
num_actions = env.action_space.n

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


### Neural Network

In [7]:
keras.backend.clear_session()

# create neural net
nn = keras.models.Sequential()

# keep this as input layer
nn.add(layers.Flatten(input_shape=(1,) + env.observation_space.shape))

# keep this as output layer
nn.add(layers.Dense(num_actions))

nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


### Optimizer

https://keras.io/optimizers/

In [0]:
# set optimizer
optimizer = keras.optimizers.SGD(lr=learning_rate)

### Memory

In [0]:
# DO NOT EDIT

# set experience buffer
mem = memory.SequentialMemory(limit=memory_limit,
                                    window_length=1)

### Policy

choose from: https://github.com/keras-rl/keras-rl/blob/master/rl/policy.py

[info on choosing](https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-7-action-selection-strategies-for-exploration-d3a97b7cceaf)

In [0]:
# set policy
pol = policy.BoltzmannQPolicy()

### Agent

choose from: https://github.com/keras-rl/keras-rl/tree/master/rl/agents

and use: https://github.com/keras-rl/keras-rl/tree/master/examples

In [0]:
# create agent
model = agents.dqn.DQNAgent(model=nn,
                            nb_actions=num_actions,
                            memory=mem,
                            gamma=gamma,
                            batch_size=batch_size,
                            nb_steps_warmup=100,
                            target_model_update=target_model_update,
                            policy=pol)

In [0]:
# DO NOT EDIT

# compile model
model.compile(optimizer,
            metrics=['mae'])

### Training

In [13]:
# DO NOT EDIT

# train
history = model.fit(env,
                  nb_steps=1000,
                  verbose=2)

Training for 1000 steps ...
  44/1000: episode: 1, duration: 0.064s, episode steps: 44, steps per second: 692, episode reward: 44.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.523 [0.000, 1.000], mean observation: 0.044 [-1.277, 1.158], loss: --, mean_absolute_error: --, mean_q: --
  65/1000: episode: 2, duration: 0.015s, episode steps: 21, steps per second: 1398, episode reward: 21.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.524 [0.000, 1.000], mean observation: 0.053 [-0.948, 1.421], loss: --, mean_absolute_error: --, mean_q: --
 156/1000: episode: 3, duration: 0.448s, episode steps: 91, steps per second: 203, episode reward: 91.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.484 [0.000, 1.000], mean observation: 0.008 [-0.852, 0.978], loss: 0.623247, mean_absolute_error: 0.687263, mean_q: 0.319911
 185/1000: episode: 4, duration: 0.097s, episode steps: 29, steps per second: 298, episode reward: 29.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.552 

### Testing

In [14]:
# DO NOT EDIT

# test
hist = model.test(env, nb_episodes=50, visualize=False)
print('mean:', np.mean(hist.history['episode_reward']))

Testing for 50 episodes ...
Episode 1: reward: 13.000, steps: 13
Episode 2: reward: 11.000, steps: 11
Episode 3: reward: 15.000, steps: 15
Episode 4: reward: 11.000, steps: 11
Episode 5: reward: 16.000, steps: 16
Episode 6: reward: 13.000, steps: 13
Episode 7: reward: 14.000, steps: 14
Episode 8: reward: 18.000, steps: 18
Episode 9: reward: 10.000, steps: 10
Episode 10: reward: 14.000, steps: 14
Episode 11: reward: 12.000, steps: 12
Episode 12: reward: 13.000, steps: 13
Episode 13: reward: 15.000, steps: 15
Episode 14: reward: 14.000, steps: 14
Episode 15: reward: 17.000, steps: 17
Episode 16: reward: 13.000, steps: 13
Episode 17: reward: 11.000, steps: 11
Episode 18: reward: 16.000, steps: 16
Episode 19: reward: 14.000, steps: 14
Episode 20: reward: 14.000, steps: 14
Episode 21: reward: 13.000, steps: 13
Episode 22: reward: 14.000, steps: 14
Episode 23: reward: 16.000, steps: 16
Episode 24: reward: 15.000, steps: 15
Episode 25: reward: 16.000, steps: 16
Episode 26: reward: 14.000, ste

### Displaying test games

The below cell will save just the neural net with its weights.

To run the test with display:

1. Go to the XWindow link, and login if you haven't already
2. In the TightVNC terminal, type and run:
   
    `run <number_repeats>`
    - replace `<number_repeats>` with however many games you want it to play (or leave it blank to run once)


In [0]:
# save model to file
nn.save('/content/models/nn.hdf5', overwrite=True)